<a href="https://colab.research.google.com/github/lisatwyw/test/blob/main/Unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
import tensorflow as tf
#tf.keras.backend.repeat_elements( KS, len(NF) )


from tensorflow.keras.layers import concatenate,add,Dense,BatchNormalization,Concatenate,Input,Dropout,Maximum,Activation,Dense,Flatten,Add,Multiply,Lambda
from tensorflow.keras.layers import MaxPooling3D,Conv3D,UpSampling3D,Conv3DTranspose
from tensorflow.keras.layers import MaxPooling2D,Conv2D,UpSampling2D,Conv2DTranspose
from tensorflow.keras.models import Model

# trivial test
o = Conv2D( 8, (1,1) )( Input( (224, 224, 3) ) )



In [92]:
def stack_bn_act( x, NF, KS  ):
  for i, ks in enumerate( KS ):
    x = Conv2D( NF, ks, padding='same' )(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
  return x

def conv_block( x, NF, KS ):

  o = Conv2D( NF, kernel_size= KS[0], padding='same' )(x)
  o = BatchNormalization()(o)
  o = Activation( 'relu' )(o)

  o = stack_bn_act(o, NF, KS[1:])
  return o

def deconv_block( x, NF, NP  ):
  o = Conv2DTranspose( NF, NP, padding='same' )(x)
  o = BatchNormalization()(o)
  o = Activation( 'relu' )(o)
  return o


AC = 'sigmoid'
BS = 8
NF = 16
NDIM=2
ks =(1,1)
ks_s=[ks]*len(nfilters)

inputs = Input( (224, 224, 3) )

# NC is len of NF
o1 = conv_block( inputs, NF, ks_s )
pool1 = MaxPooling2D()( o1 )

o2 = conv_block( o1,  NF*2, ks_s )
pool2 = MaxPooling2D()( o2 )

o3 = conv_block( o1, NF*4, ks_s )
pool3 = MaxPooling2D()( o3 )

o4 = conv_block( o1, NF*8, ks_s )
pool4 = MaxPooling2D()( o4 )

o5 = conv_block( o1, NF*16, ks_s )
pool5 = MaxPooling2D()( o5 )

NP=2

o6 = conv_block( Concatenate()( [deconv_block( o5, NF*8, NP ), o4]), NF*8, KS )
o7 = conv_block( Concatenate()( [deconv_block( o6, NF*4, NP ), o3]), NF*4, KS )

o8 = conv_block( Concatenate()( [deconv_block( o7, NF*2, NP ), o2]), NF*2, KS )
o9 = conv_block( Concatenate()( [deconv_block( o8, NF, NP ), o1]), NF, KS )

outputs = Activation( AC )( Conv2D( 2, ks )( o9 ) )
model = Model(inputs, outputs=outputs )




(None, 224, 224, 16) << (None, 224, 224, 128)


In [93]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_60 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_629 (Conv2D)            (None, 224, 224, 16  64          ['input_60[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_648 (Batch  (None, 224, 224, 16  64         ['conv2d_629[0][0]']             
 Normalization)                 )                                                             